In [1]:
from alpaca.data.historical import NewsClient
from alpaca.data.requests import StockBarsRequest

In [2]:
import os
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
from alpaca.data.historical import  StockHistoricalDataClient,NewsClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.common.enums import Sort
from datetime import datetime
from data_pipeline.stage_01.historical_data import add_technical_indicators,supertrend
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch
load_dotenv()

api = os.getenv('ALPACA_API_KEY')
secret = os.getenv('ALPACA_SECRET_KEY')


def get_sentiment_score(headline):
    model_name = "ProsusAI/finbert"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    inputs = tokenizer(headline, max_length=512, truncation=True, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1).detach().numpy()[0]
    label = model.config.id2label
    predicted_class = label[int(torch.argmax(outputs.logits).numpy())]
    

    return predicted_class


def create_data(data, max_lag = 15):
  max_lag = max_lag  

  for i in range(1, max_lag + 1):
    data[f'Close_lag{i}'] = data['Close'].shift(i)


  return data

def get_data(symbol ='AAPL', api = os.getenv('ALPACA_API_KEY') ,secret = os.getenv('ALPACA_SECRET_KEY')):
    # Get the current date and time
    now = datetime.now()

    # Format the datetime object into YYYYMMDD format
    formatted_date = now.strftime('%Y%m%d')

    # Convert the formatted string back into a datetime object
    formatted_datetime = datetime.strptime(formatted_date, '%Y%m%d')

    client = StockHistoricalDataClient(api, secret)
    params = StockBarsRequest(
        symbol_or_symbols=symbol,timeframe=TimeFrame.Day, start=datetime(2024,6, 1), end=formatted_datetime, limit=100,sort=Sort.DESC,feed='iex'
        
    )

    data = client.get_stock_bars(params)
    data = dict(data)
    check = []
    for i in data['data'][symbol]:
        check.append(dict(i))
    df = pd.DataFrame(check)
    df.sort_values(by='timestamp',inplace=True)
    # Convert 'timestamp' to datetime and extract only the date
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.date

    # Rename columns
    df.rename(columns={
        'timestamp': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'volume':'Volume'
    }, inplace=True)

    # Keep only the required columns
    df = df[['Date', 'Open', 'High', 'Low', 'Close','Volume']]
    
    df = supertrend(df , factor=3, atr_period=10)
    df = add_technical_indicators(df)
    df = df.drop(columns=['Supertrend','UpperBand', 'LowerBand', 'Uptrend',
        'Downtrend','Direction'])
    df.dropna(inplace=True)
    df.set_index('Date', inplace=True)
    

    return df

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = get_data()

In [4]:
df

,Open,High,Low,Close,Volume,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,DMP_14,DMN_14,EMA_20,BBL_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,OBV,STOCHk_14_3_3,STOCHd_14_3_3
Date,,,,,,,,,,,,,,,,,,,,
2025-01-23,224.74,227.010,222.300,223.570,1397373.0,28.925596,-6.769356,-2.079087,-4.690269,37.235528,15.817075,38.333743,236.287117,217.996456,263.755044,18.996760,0.121803,-2667784.0,12.194218,9.068292
2025-01-24,225.10,225.615,221.420,222.850,1287472.0,28.351972,-7.017768,-1.861999,-5.155769,37.759424,14.847319,37.269610,235.007392,216.008167,262.602333,19.470599,0.146839,-3955256.0,13.194973,11.657755
2025-01-27,224.02,232.140,224.020,229.980,2657424.0,40.859495,-6.563643,-1.126299,-5.437344,36.135649,21.913169,32.515252,234.528592,215.964848,259.825652,18.437024,0.319537,-1297832.0,21.463757,15.617649
2025-01-28,230.85,240.190,230.850,238.230,2008290.0,51.425774,-5.474928,-0.030067,-5.444861,33.152154,29.381300,28.249953,234.881107,217.196373,256.502127,16.595262,0.535128,710458.0,40.786593,25.148441
2025-01-29,234.15,239.850,234.050,239.315,1126638.0,52.624584,-4.473001,0.777488,-5.250489,30.471699,27.198592,26.151292,235.303383,218.307358,253.759142,15.019827,0.592569,1837096.0,64.013175,42.087842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-23,206.00,207.940,202.830,204.330,1708419.0,48.858912,-6.085447,1.447069,-7.532515,27.157729,25.376640,27.670070,202.873014,173.949268,231.871232,28.545607,0.524511,-16971969.0,59.177089,51.237813
2025-04-24,205.05,208.800,202.965,208.200,1120255.0,51.311415,-4.945157,2.069887,-7.015044,25.429795,24.912701,26.463335,203.380346,174.425474,230.075026,27.515196,0.606915,-15851714.0,74.636864,60.886309
2025-04-25,206.44,209.430,206.440,209.180,964315.0,51.939942,-3.917236,2.478246,-6.395482,23.753730,24.822398,25.841417,203.932694,175.266952,227.777548,26.056972,0.645832,-14887399.0,87.083954,73.632636


In [5]:
df.iloc[-1]

Open             2.086800e+02
High             2.121100e+02
Low              2.083900e+02
Close            2.116500e+02
Volume           2.692020e+05
RSI_14           5.364364e+01
MACD_12_26_9    -2.113873e+00
MACDh_12_26_9    2.881282e+00
MACDs_12_26_9   -4.995155e+00
ADX_14           2.074106e+01
DMP_14           2.540992e+01
DMN_14           2.420134e+01
EMA_20           2.051974e+02
BBL_20_2.0       1.764550e+02
BBU_20_2.0       2.247740e+02
BBB_20_2.0       2.408550e+01
BBP_20_2.0       7.283884e-01
OBV             -1.378720e+07
STOCHk_14_3_3    9.405425e+01
STOCHd_14_3_3    9.087222e+01
Name: 2025-04-29, dtype: float64

In [6]:
create_data(df).iloc[-1]

Open             2.086800e+02
High             2.121100e+02
Low              2.083900e+02
Close            2.116500e+02
Volume           2.692020e+05
RSI_14           5.364364e+01
MACD_12_26_9    -2.113873e+00
MACDh_12_26_9    2.881282e+00
MACDs_12_26_9   -4.995155e+00
ADX_14           2.074106e+01
DMP_14           2.540992e+01
DMN_14           2.420134e+01
EMA_20           2.051974e+02
BBL_20_2.0       1.764550e+02
BBU_20_2.0       2.247740e+02
BBB_20_2.0       2.408550e+01
BBP_20_2.0       7.283884e-01
OBV             -1.378720e+07
STOCHk_14_3_3    9.405425e+01
STOCHd_14_3_3    9.087222e+01
Close_lag1       2.100800e+02
Close_lag2       2.091800e+02
Close_lag3       2.082000e+02
Close_lag4       2.043300e+02
Close_lag5       1.995600e+02
Close_lag6       1.932500e+02
Close_lag7       1.968900e+02
Close_lag8       1.942700e+02
Close_lag9       2.021400e+02
Close_lag10      2.026100e+02
Close_lag11      1.981500e+02
Close_lag12      1.906300e+02
Close_lag13      1.984600e+02
Close_lag1

In [7]:
from alpaca.data.historical import NewsClient
from alpaca.data.requests import NewsRequest

In [8]:
nclient = NewsClient(api,secret, raw_data=True)
params = NewsRequest(symbol_or_symbols = 'AAPL', limit=50)
data = pd.DataFrame(nclient.get_news(params)['news'])
data['Time'] = pd.DatetimeIndex(data['updated_at'])
data =data.drop(columns=['updated_at','created_at','content','images','symbols','url','author','source','summary','id'])
data["Date"] = data["Time"].dt.date
data = data.groupby("Date")["headline"].apply(lambda x: " | ".join(x)).reset_index()
data.set_index('Date',inplace=True)
df.loc[df.index[-1:], 'sentiment_class'] = get_sentiment_score(data['headline'].iloc[0]) 

In [9]:
df.iloc[-1:]

,Open,High,Low,Close,Volume,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,...,Close_lag7,Close_lag8,Close_lag9,Close_lag10,Close_lag11,Close_lag12,Close_lag13,Close_lag14,Close_lag15,sentiment_class
Date,,,,,,,,,,,,,,,,,,,,,
2025-04-29,208.68,212.11,208.39,211.65,269202.0,53.643642,-2.113873,2.881282,-4.995155,20.741057,...,196.89,194.27,202.14,202.61,198.15,190.63,198.46,172.8,181.58,neutral


In [10]:
X =df.iloc[-1:]

In [11]:
df

,Open,High,Low,Close,Volume,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,...,Close_lag7,Close_lag8,Close_lag9,Close_lag10,Close_lag11,Close_lag12,Close_lag13,Close_lag14,Close_lag15,sentiment_class
Date,,,,,,,,,,,,,,,,,,,,,
2025-01-23,224.74,227.010,222.300,223.570,1397373.0,28.925596,-6.769356,-2.079087,-4.690269,37.235528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-24,225.10,225.615,221.420,222.850,1287472.0,28.351972,-7.017768,-1.861999,-5.155769,37.759424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-27,224.02,232.140,224.020,229.980,2657424.0,40.859495,-6.563643,-1.126299,-5.437344,36.135649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-28,230.85,240.190,230.850,238.230,2008290.0,51.425774,-5.474928,-0.030067,-5.444861,33.152154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-29,234.15,239.850,234.050,239.315,1126638.0,52.624584,-4.473001,0.777488,-5.250489,30.471699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-23,206.00,207.940,202.830,204.330,1708419.0,48.858912,-6.085447,1.447069,-7.532515,27.157729,...,198.15,190.63,198.46,172.80,181.58,188.50,203.01,223.80,223.20,NaN
2025-04-24,205.05,208.800,202.965,208.200,1120255.0,51.311415,-4.945157,2.069887,-7.015044,25.429795,...,202.61,198.15,190.63,198.46,172.80,181.58,188.50,203.01,223.80,NaN
2025-04-25,206.44,209.430,206.440,209.180,964315.0,51.939942,-3.917236,2.478246,-6.395482,23.753730,...,202.14,202.61,198.15,190.63,198.46,172.80,181.58,188.50,203.01,NaN


In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Load FinBERT model and tokenizer
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

inputs = tokenizer('HC Wainwright & Co. Maintains Buy on Uranium Royalty, Lowers Price Target to $4.5', max_length=512, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

# Get probabilities
probs = torch.nn.functional.softmax(outputs.logits, dim=-1).detach().numpy()[0]
# FinBERT classes: negative (0), neutral (1), positive (2)
sentiment_score = probs[2] - probs[0]  # positive - negative
print(probs)
print(model.config.id2label)

[0.3557707  0.20528969 0.43893954]
{0: 'positive', 1: 'negative', 2: 'neutral'}


In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Load FinBERT model and tokenizer
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

inputs = tokenizer('FUJIFILM Diosynth Biotechnologies Ink 10-Year Manufacturing Supply Agreement Worth $3B With Regeneron Pharmaceuticals', max_length=512, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

# Get probabilities
probs = torch.nn.functional.softmax(outputs.logits, dim=-1).detach().numpy()[0]
# FinBERT classes: negative (0), neutral (1), positive (2)
sentiment_score = probs[2] - probs[0]  # positive - negative
print(probs)
print(model.config.id2label)

[0.92157775 0.01051987 0.06790239]
{0: 'positive', 1: 'negative', 2: 'neutral'}


In [14]:
df.loc[df.index[-1], 'headline'] = data['headline'].iloc[0]

In [15]:
df.loc[df.index[-1]]

Open                                                          208.68
High                                                          212.11
Low                                                           208.39
Close                                                         211.65
Volume                                                      269202.0
RSI_14                                                     53.643642
MACD_12_26_9                                               -2.113873
MACDh_12_26_9                                               2.881282
MACDs_12_26_9                                              -4.995155
ADX_14                                                     20.741057
DMP_14                                                     25.409923
DMN_14                                                     24.201336
EMA_20                                                    205.197375
BBL_20_2.0                                                176.454995
BBU_20_2.0                        

In [16]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import OrderRequest, GetOrdersRequest,LimitOrderRequest,LimitOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce, OrderType

In [44]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import OrderRequest,GetAssetsRequest,GetCalendarRequest,StopLossRequest, LimitOrderRequest,CancelOrderResponse,StopOrderRequest,GetOrdersRequest
from alpaca.trading.enums import OrderSide,OrderType,TimeInForce,OrderStatus

tcliet = TradingClient(api,secret)

params = OrderRequest(symbol='AAPL',side=OrderSide.BUY,type=OrderType.MARKET,qty=10, time_in_force=TimeInForce.DAY)
tcliet.submit_order(params)

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'canceled_at': None,
    'client_order_id': '99239b92-145a-4f80-a522-9b2f9baec006',
    'created_at': datetime.datetime(2025, 5, 1, 2, 7, 53, 165595, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 5, 1, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('30fa263d-6c3c-45d7-8b7b-a44fdd94c5b2'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '10',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.

In [57]:
dict(tcliet.get_all_positions()[0])['symbol']

'MSFT'

In [46]:
tcliet.get_orders()


[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
     'canceled_at': None,
     'client_order_id': '99239b92-145a-4f80-a522-9b2f9baec006',
     'created_at': datetime.datetime(2025, 5, 1, 2, 7, 53, 165595, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'expires_at': datetime.datetime(2025, 5, 1, 20, 0, tzinfo=TzInfo(UTC)),
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('30fa263d-6c3c-45d7-8b7b-a44fdd94c5b2'),
     'legs': None,
     'limit_price': None,
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.MARKET: 'market'>,
     'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
     'qty': '10',
     'ratio_qty': None,
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
 

In [43]:
tcliet.get_open_position(symbol_or_asset_id='MSFT')


{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b6d1aa75-5c9c-4353-a305-9e2caa1925ab'),
    'asset_marginable': True,
    'avg_entry_price': '393.3495',
    'avg_entry_swap_rate': None,
    'change_today': '0',
    'cost_basis': '3933.495',
    'current_price': '395.26',
    'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
    'lastday_price': '395.26',
    'market_value': '3952.6',
    'qty': '10',
    'qty_available': '10',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'MSFT',
    'unrealized_intraday_pl': '0',
    'unrealized_intraday_plpc': '0',
    'unrealized_pl': '19.105',
    'unrealized_plpc': '0.0048570037587438',
    'usd': None}

In [58]:
tcliet.close_position('MSFT')

APIError: {"code":40310000,"existing_order_id":"3cba6f36-5e44-476d-9ea9-daae779e99a0","message":"potential wash trade detected. use complex orders","reject_reason":"opposite side market/stop order exists"}

In [61]:
tcliet.cancel_orders()

[{   'body': None,
     'id': UUID('402a24d5-d6d3-425c-9b1d-2259c75e565a'),
     'status': 200},
 {   'body': None,
     'id': UUID('4c25be0f-cc3a-4f6b-b7a2-c8422e7a0eca'),
     'status': 200},
 {   'body': None,
     'id': UUID('7fd87161-3481-4c0b-a6d6-c0aa5c27691a'),
     'status': 200},
 {   'body': None,
     'id': UUID('9d5140ce-1786-4195-866b-5b1da2902dc2'),
     'status': 200}]

In [60]:
tcliet.get_all_positions()

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b6d1aa75-5c9c-4353-a305-9e2caa1925ab'),
     'asset_marginable': True,
     'avg_entry_price': '393.3495',
     'avg_entry_swap_rate': None,
     'change_today': '0',
     'cost_basis': '3933.495',
     'current_price': '395.26',
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'lastday_price': '395.26',
     'market_value': '3952.6',
     'qty': '10',
     'qty_available': '10',
     'side': <PositionSide.LONG: 'long'>,
     'swap_rate': None,
     'symbol': 'MSFT',
     'unrealized_intraday_pl': '0',
     'unrealized_intraday_plpc': '0',
     'unrealized_pl': '19.105',
     'unrealized_plpc': '0.0048570037587438',
     'usd': None}]

In [23]:
df.iloc[-1:,:-1]

,Open,High,Low,Close,Volume,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,...,Close_lag7,Close_lag8,Close_lag9,Close_lag10,Close_lag11,Close_lag12,Close_lag13,Close_lag14,Close_lag15,sentiment_class
Date,,,,,,,,,,,,,,,,,,,,,
2025-04-29,208.68,212.11,208.39,211.65,269202.0,53.643642,-2.113873,2.881282,-4.995155,20.741057,...,196.89,194.27,202.14,202.61,198.15,190.63,198.46,172.8,181.58,neutral


In [24]:
path = r'C:\Users\sachi\Documents\Researchcode\saved_models\MSFT_lag_15_model.pkl'
scaler_path = r'C:\Users\sachi\Documents\Researchcode\artifact\scaler_AMZN_15.pkl'
encoder_path = r'C:\Users\sachi\Documents\Researchcode\artifact\encoder_MSFT_15.pkl'

In [25]:
import joblib
model = joblib.load(path)
scaler = joblib.load(scaler_path)
encoder = joblib.load(encoder_path)

In [26]:
X_test = df.iloc[-1:,:-1].copy()

In [27]:
X_test['sentiment_class'][0]

C:\Users\sachi\AppData\Local\Temp\ipykernel_57084\2331438617.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X_test['sentiment_class'][0]


'neutral'

In [28]:
X_test

,Open,High,Low,Close,Volume,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,...,Close_lag7,Close_lag8,Close_lag9,Close_lag10,Close_lag11,Close_lag12,Close_lag13,Close_lag14,Close_lag15,sentiment_class
Date,,,,,,,,,,,,,,,,,,,,,
2025-04-29,208.68,212.11,208.39,211.65,269202.0,53.643642,-2.113873,2.881282,-4.995155,20.741057,...,196.89,194.27,202.14,202.61,198.15,190.63,198.46,172.8,181.58,neutral


In [29]:
encoder.classes_

array(['negative', 'neutral', 'positive'], dtype=object)

In [30]:
encoder.get_metadata_routing()

{}

In [31]:
X_test['sentiment_class'] = encoder.transform(X_test['sentiment_class'])

In [32]:
scaler.feature_names_in_

array(['Close', 'High', 'Low', 'Open', 'Volume', 'sentiment_class',
       'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9',
       'ADX_14', 'DMP_14', 'DMN_14', 'EMA_20', 'BBL_20_2.0', 'BBU_20_2.0',
       'BBB_20_2.0', 'BBP_20_2.0', 'OBV', 'STOCHk_14_3_3',
       'STOCHd_14_3_3', 'Close_lag1', 'Close_lag2', 'Close_lag3',
       'Close_lag4', 'Close_lag5', 'Close_lag6', 'Close_lag7',
       'Close_lag8', 'Close_lag9', 'Close_lag10', 'Close_lag11',
       'Close_lag12', 'Close_lag13', 'Close_lag14', 'Close_lag15'],
      dtype=object)

In [34]:
X_test = X_test[['Close', 'High', 'Low', 'Open', 'Volume', 'sentiment_class',
       'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9',
       'ADX_14', 'DMP_14', 'DMN_14', 'EMA_20', 'BBL_20_2.0', 'BBU_20_2.0',
       'BBB_20_2.0', 'BBP_20_2.0', 'OBV', 'STOCHk_14_3_3',
       'STOCHd_14_3_3', 'Close_lag1', 'Close_lag2', 'Close_lag3',
       'Close_lag4', 'Close_lag5', 'Close_lag6', 'Close_lag7',
       'Close_lag8', 'Close_lag9', 'Close_lag10', 'Close_lag11',
       'Close_lag12', 'Close_lag13', 'Close_lag14', 'Close_lag15']]

In [35]:
X_test_scaled = scaler.transform(X_test)

In [36]:
model.predict(X_test_scaled)

array([0])